# Gaode Maps Weather Information API Test Cases

This notebook contains automated test cases for the Gaode Maps Weather Information API.

In [ ]:
import requests
import json
from datetime import datetime

# Global variables
API_KEY = 'your_api_key_here'  # Replace with actual API key
BASE_URL = 'http://restapi.amap.com/v3/weather/weatherInfo'

# Test cities
VALID_CITY = 'Beijing'
INVALID_CITY = 'NonExistentCity'

def make_request(params):
    """Helper function to make API requests"""
    response = requests.get(BASE_URL, params=params)
    return response.json()

def validate_base_response(response):
    """Validate basic response structure"""
    assert isinstance(response, dict), 'Response should be a dictionary'
    assert 'lives' in response or 'forecasts' in response, 'Response should contain lives or forecasts'

def validate_weather_data(weather_data):
    """Validate weather data fields"""
    required_fields = ['city', 'weather', 'temperature', 'winddirection', 'windpower', 'humidity', 'reporttime']
    for field in required_fields:
        assert field in weather_data, f'Weather data should contain {field}'
        assert weather_data[field] is not None, f'{field} should not be None'

## Main Flow Test Cases

In [ ]:
def test_basic_weather_query():
    """Test basic weather query with required parameters"""
    print('Running test_basic_weather_query...')
    
    # Test setup
    params = {
        'key': API_KEY,
        'city': VALID_CITY
    }
    
    # Execute request
    response = make_request(params)
    
    # Assertions
    validate_base_response(response)
    assert 'lives' in response, 'Basic query should return lives data'
    assert len(response['lives']) > 0, 'Lives array should not be empty'
    validate_weather_data(response['lives'][0])
    
    print('✓ test_basic_weather_query passed')

# Run the test
test_basic_weather_query()

## Branch Flow Test Cases

In [ ]:
def test_weather_extensions():
    """Test weather query with different extensions"""
    print('Running test_weather_extensions...')
    
    # Test base extension
    base_params = {
        'key': API_KEY,
        'city': VALID_CITY,
        'extensions': 'base'
    }
    base_response = make_request(base_params)
    validate_base_response(base_response)
    assert 'lives' in base_response, 'Base extension should return lives data'
    
    # Test all extension
    all_params = {
        'key': API_KEY,
        'city': VALID_CITY,
        'extensions': 'all'
    }
    all_response = make_request(all_params)
    validate_base_response(all_response)
    assert 'forecasts' in all_response, 'All extension should return forecasts data'
    
    # Validate forecast data
    forecasts = all_response['forecasts']
    assert len(forecasts) > 0, 'Forecasts array should not be empty'
    assert 'casts' in forecasts[0], 'Forecast should contain casts'
    assert len(forecasts[0]['casts']) > 0, 'Casts array should not be empty'
    
    print('✓ test_weather_extensions passed')

# Run the test
test_weather_extensions()

## Exception Flow Test Cases

In [ ]:
def test_invalid_parameters():
    """Test API behavior with invalid parameters"""
    print('Running test_invalid_parameters...')
    
    # Test invalid API key
    invalid_key_params = {
        'key': 'invalid_key',
        'city': VALID_CITY
    }
    invalid_key_response = make_request(invalid_key_params)
    assert 'status' in invalid_key_response, 'Error response should contain status'
    assert invalid_key_response['status'] != '1', 'Invalid key should return error status'
    
    # Test invalid city
    invalid_city_params = {
        'key': API_KEY,
        'city': INVALID_CITY
    }
    invalid_city_response = make_request(invalid_city_params)
    assert 'status' in invalid_city_response, 'Error response should contain status'
    assert invalid_city_response['status'] != '1', 'Invalid city should return error status'
    
    # Test invalid extension
    invalid_ext_params = {
        'key': API_KEY,
        'city': VALID_CITY,
        'extensions': 'invalid'
    }
    invalid_ext_response = make_request(invalid_ext_params)
    assert 'status' in invalid_ext_response, 'Error response should contain status'
    assert invalid_ext_response['status'] != '1', 'Invalid extension should return error status'
    
    print('✓ test_invalid_parameters passed')

# Run the test
test_invalid_parameters()

## Run All Tests

In [ ]:
def run_all_tests():
    """Run all test cases"""
    print('Starting test suite...')
    print('-' * 50)
    
    try:
        test_basic_weather_query()
        print('-' * 50)
        test_weather_extensions()
        print('-' * 50)
        test_invalid_parameters()
        print('-' * 50)
        print('All tests completed successfully!')
    except AssertionError as e:
        print(f'Test failed: {str(e)}')
    except Exception as e:
        print(f'Unexpected error: {str(e)}')

# Run all tests
run_all_tests()